# Лабораторная работа №2. Методы поиска.
## Выполнил студент группы БСТ1903 Талантов О.А. Вариант 18

### Задание 1

1. Реализовать методы поиска в соответствии с заданием.
2. Организовать генерацию начального набора случайных данных.
3. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов.
4. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.


#### Импорты

In [22]:
import random
import time
from __future__ import print_function

#### Генерация случайных значений

In [23]:
def random_matrix(m = 50, n = 50, min_limit = -250, max_limit = 1018):
    return [[random.randint(min_limit, max_limit) for _ in range(n)] for _ in range(m)]

### Бинарный поиск

In [24]:
class BinarySearchMap:
    def __init__(self):
        self.data = [] # хранилище (key, value) значений

    def search(self, key):
        """ Поиск индекса (во всех случаях лучше левосторонний,
            чтоб insert вставлял по убыванию) """
        l = 0
        r = len(self.data)
        while l < r:
            m = (l + r) // 2
            if self.data[m][0] < key:
                l = m + 1
            else:
                r = m
        return l    
        
    def __setitem__(self, key, value):
        """ Добавить элемент """
        index = self.search(key)
        # если ключ уже есть в таблице, то надо заменить значение
        if index < len(self.data) and self.data[index][0] == key:
            self.data[index] = (key, value)
        else:
            # иначе добавляем новую запись
            self.data.insert(index, (key, value))
    
    def __delitem__(self, key):
        """ Удалить элемент """
        index = self.search(key)
        self.data.pop(index)
    
    def __getitem__(self, key):
        """ Получить элемент """
        index = self.search(key)
        found_key, val = self.data[index]
        # если найденный индекс выдает запрашиваемый ключ
        if found_key == key:
            return val
        raise KeyError()


### Фибоначчиев поиск

In [25]:
fib_c = [0, 1]
def fib(n):
    if len(fib_c) - 1 < n:
        fib_c.append(fib(n - 1) + fib(n - 2))
    return fib_c[n]

class FibonacciMap(BinarySearchMap):
    def search(self, key):
        m = 0 
        while fib(m) < len(self.data): 
            m += 1
        offset = 0
        while fib(m) > 1:
            i = min(offset + fib(m - 1), len(self.data) - 1)
            if key > self.data[i][0]:
                offset = i
            elif key == self.data[i][0]:
                return i
            m -= 1
        if len(self.data) and self.data[offset][0] < key:
            return offset + 1
        return 0

### Интерполяционный поиск

In [26]:
def nearest_mid(input_list, lower_bound_index, upper_bound_index, search_value):
    return lower_bound_index + \
        (upper_bound_index - lower_bound_index) * \
        (search_value - input_list[lower_bound_index]) // \
        (input_list[upper_bound_index][0] - input_list[lower_bound_index][0])

class InterpolateMap(BinarySearchMap):
    def interpolation_search(self, term):
        size_of_list = len(self.data) - 1

        index_of_first_element = 0
        index_of_last_element = size_of_list

        while index_of_first_element <= index_of_last_element:
            mid_point = nearest_mid(self.data, index_of_first_element, index_of_last_element, term)

            if mid_point > index_of_last_element or mid_point < index_of_first_element:
                return None

            if self.data[mid_point][0] == term:
                return mid_point

            if term > self.data[mid_point][0]:
                index_of_first_element = mid_point + 1
            else:
                index_of_last_element = mid_point - 1

        if index_of_first_element > index_of_last_element:
            return None

### Бинарное дерево

In [27]:
class Tree:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.left = self.right = None
        
class BinaryTreeMap:
    root = None
    
    def insert(self, tree, key, value):
        if tree is None:
            return Tree(key, value)
        if tree.key > key:
            tree.left = self.insert(tree.left, key, value)
        elif tree.key < key:
            tree.right = self.insert(tree.right, key, value)
        else:
            tree.value = value
        return tree
            
    def search(self, tree, key):
        if tree is None or tree.key == key:
            return tree
        if tree.key > key:
            return self.search(tree.left, key)
        return self.search(tree.right, key)
        
    def __getitem__(self, key):
        tree = self.search(self.root, key)
        if tree is not None:
            return tree.value
        raise KeyError()
    
    def __setitem__(self, key, value):
        if self.root is None:
            self.root = self.insert(self.root, key, value)
        else: self.insert(self.root, key, value)

### Задание 2


### Простое рехеширование

In [28]:
class HashMap:
    def __init__(self):
        self.size = 0
        self.data = []
        self._resize()
    
    def _hash(self, key, i):
        return (hash(key) + i) % len(self.data)
        
    def _find(self, key):
        i = 0;
        index = self._hash(key, i);
        while self.data[index] is not None and self.data[index][0] != key:
            i += 1
            index = self._hash(key, i);
        return index;
    
    def _resize(self):
        temp = self.data
        self.data = [None] * (2*len(self.data) + 1)
        for item in temp:
            if item is not None:
                self.data[self._find(item[0])] = item
    
    def __setitem__(self, key, value):
        if self.size + 1 > len(self.data) // 2:
            self._resize()
        index = self._find(key)
        if self.data[index] is None:  
            self.size += 1
        self.data[index] = (key, value)
    
    def __getitem__(self, key):
        index = self._find(key)
        if self.data[index] is not None:
            return self.data[index][1]
        raise KeyError()

### Рехеширование с помощью всевдослучайных чисел

In [29]:
class RandomHashMap(HashMap):
    _rand_c = [5323]
    
    def _rand(self, i):
        if len(self._rand_c) - 1 < i:
            self._rand_c.append(self._rand(i - 1))
        return (123456789 * self._rand_c[i] + 987654321) % 65546
        
    def _hash(self, key, i):
        return (hash(key) + self._rand(i)) % len(self.data)

### Метод цепочек

In [30]:
class ChainMap:
    def __init__(self):
        self.size = 0
        self.data = []
        self._resize()
    
    def _hash(self, key):
        return hash(key) % len(self.data)
    
    def _insert(self, index, item):
        if self.data[index] is None:
            self.data[index] = [item]
            return True
        else:
            for i, item_ in enumerate(self.data[index]):
                if item_[0] == item[0]:
                    self.data[index][i] = item
                    return False
            self.data[index].append(item)
            return True
    
    def _resize(self):
        temp = self.data
        self.data = [None] * (2*len(self.data) + 1)
        for bucket in temp:
            if bucket is not None:
                for key, value in bucket:
                    self._insert(self._hash(key), (key, value))
    
    def __setitem__(self, key, value):
        if self.size + 1 > len(self.data) // 1.5:
            self._resize()
        if self._insert(self._hash(key), (key, value)):  
            self.size += 1
    
    def __getitem__(self, key):
        index = self._hash(key)
        if self.data[index] is not None:
            for key_, value in self.data[index]:
                if key_ == key:
                    return value
        raise KeyError()

### Сравнение алгоритмов

In [31]:
алгоритмы = {
    'Бинарный поиск': BinarySearchMap,
    'Фибоначчиева поиск': FibonacciMap,
    'Интерполяционный поиск': InterpolateMap,
    'Бинарное дерево': BinaryTreeMap,
    'Простое рехэширование': HashMap,
    'Рехэширование с помощью псевдослучайных чисел': RandomHashMap,
    'Метод цепочек': ChainMap,
    'Стандартная функция поиска': dict
}


затраченное_время = {}
тестовые_набор = random_matrix(50, 1000)
for имя_алгоритма, Таблица in алгоритмы.items():
    копия_наборов = тестовые_набор.copy()
    время_начало = time.perf_counter()
    for набор in копия_наборов:
        таблица = Таблица()
        for значение, ключ in enumerate(набор):
            таблица[ключ] = значение
            assert таблица[ключ] == значение, f'Найденный элемент не соответствует записанному'
    время_конца = time.perf_counter()
    затраченное_время[имя_алгоритма] = (время_конца - время_начало) / len(тестовые_набор)

отсортированная_таблица_затраченного_времени = sorted(затраченное_время.items(), key=lambda kv: kv[1])
tabulate(отсортированная_таблица_затраченного_времени, headers=['Алгоритм','Время'], tablefmt='html', showindex="always")

NameError: name 'tabulate' is not defined

### Задание 3

Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям
Написать программу,  которая находит хотя бы один способ решения задач.



In [ ]:
board = [[0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1],
         [0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0]]

def clearBoard(board):
    for i in range(8):
        for j in range(8):
            board[i][j] = 0

def printBoard(board):
    for i in range(8):
        for j in range(8):
            if board[i][j] == 1:
                print("F", end="")
            else:
                print("X", end="")
        print("")
    print("")

def checkColsOK(board):
    for i in range(8):
        sum = 0
        for j in range(8):
            sum += board[j][i]
        if sum > 1:
            return 0




def checkRowsOK(board):
    for i in range(8):
        sum = 0
        for j in range (8):
            sum += board[i][j]
        if sum > 1:
            return 0


def checkDiagsOK(board):

    counter = 8
    sum = 0

    for i in range(8):
        x = i
        y = 0
        for j in range(counter):
            sum += board[y][x]
            x += 1
            y +=1
        counter -= 1

        if sum > 1:
            return 0
        sum = 0


    counter = 8
    sum = 0

    for i in range(8):
        x = i
        y = 0
        for j in range(counter):
            sum += board[x][y]
            x += 1
            y +=1
        counter -= 1


        if sum > 1:
            return 0
        sum = 0


    counter = 8
    sum = 0

    for i in reversed(range(8)):
        x = i
        y = 0
        for j in range(counter):
            sum += board[x][y]
            x -= 1
            y += 1
        counter -= 1


        if sum > 1:
            return 0
        sum = 0


    counter = 8
    sum = 0

    for i in range(8):
        x = 7
        y = i
        for j in range(counter):
            sum += board[x][y]
            x -= 1
            y += 1
        counter -= 1

        

        if sum > 1:
            return 0
        sum = 0

def addFerz(board, col):

    row = 0

    for row in range(8):
        board[row][col] = 1
        if (checkRowsOK(board) != 0 and checkDiagsOK(board) != 0):
            if col == 7:
                printBoard(board)
            else:
                addFerz(board, col + 1)
        board[row][col] = 0


clearBoard(board)
addFerz(board, 0)

### Вывод
В ходе лабораторной работы мы научились реализовывать методы поиска, организовывать генерацию начального набора случайных данных, а также добавили реализацию добавления, поиска и удаления элементов, оценили время работы каждого алгоритма поиска и сравнили его со временем работы стандартной функции поиска, используемой в языке программирования Python 3. А также реализовали один из вариантов решения задачи о восьми ферзях.